In [1]:
# Import some necessary library

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
import tqdm

In [2]:
# Build the 18-layer ResNet model
class BasicBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(residual)
        out = F.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out


def resnet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [3]:
# Initialize two ResNet 18 models
device = "cuda" if torch.cuda.is_available() else "cpu"
model_SGD = resnet18().to(device)
model_ADAM = resnet18().to(device)

In [4]:
# Load CIFAR 10 Datasets

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [17]:
# Train function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    model.train()
    for X, y in tqdm.tqdm(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= num_batches
    correct /= size
    print(f"Avg Train loss: {loss:>8f} \n")
    return train_loss, correct

In [18]:
# Test function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss, correct

In [19]:
import torch.optim as optim

# Define the loss function 
criterion = nn.CrossEntropyLoss()

# set the optimizer as SGD with Momentum 
# Please finish this part
optimizer_SGD = optim.SGD(model_SGD.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Train model_SGD by SGD with Momentum optimization algorith
# Please add code to finish this part.
train_loss_SGD, train_acc_SGD = [], []
test_loss_SGD, test_acc_SGD = [], []

for epoch in range(30):
    train_loss, train_acc = train(trainloader, model_SGD, criterion, optimizer_SGD)
    train_loss_SGD.append(train_loss)
    train_acc_SGD.append(train_acc)
    test_loss, test_acc = test(testloader, model_SGD, criterion)
    test_loss_SGD.append(test_loss)
    test_acc_SGD.append(test_acc)


In [ ]:
# Repeat the training and testing procedure for model_ADAM
# Visualize the results and save the images. Add the images to your assignment solution.
optimizer_ADAM = optim.Adam(model_ADAM.parameters(), lr=0.01)

train_loss_ADAM, train_acc_ADAM = [], []
test_loss_ADAM, test_acc_ADAM = [], []

for epoch in range(30):
    train_loss, train_acc = train(trainloader, model_ADAM, criterion, optimizer_ADAM)
    train_loss_ADAM.append(train_loss)
    train_acc_ADAM.append(train_acc)
    test_loss, test_acc = test(testloader, model_ADAM, criterion)
    test_loss_ADAM.append(test_loss)
    test_acc_ADAM.append(test_acc)

In [ ]:
# Visualize the results (You can refer to other tutorial notebooks)
import matplotlib.pyplot as plt

## Add codes to finish this part and save the images. Add the images to your assignment solution.
plt.figure(figsize=(12, 5))
plt.subplot(2, 2, 1)
plt.plot(train_loss_SGD, label='Train Loss SGD')
plt.plot(train_loss_ADAM, label='Train Loss ADAM')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Train Loss vs Epochs')

plt.subplot(2, 2, 2)
plt.plot(test_loss_SGD, label='Test Loss SGD')
plt.plot(test_loss_ADAM, label='Test Loss ADAM')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Test Loss vs Epochs')

plt.subplot(2, 2, 3)
plt.plot(train_acc_SGD, label='Train Accuracy SGD')
plt.plot(train_acc_ADAM, label='Train Accuracy ADAM')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Train Accuracy vs Epochs')

plt.subplot(2, 2, 4)
plt.plot(test_acc_SGD, label='Test Accuracy SGD')
plt.plot(test_acc_ADAM, label='Test Accuracy ADAM')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Test Accuracy vs Epochs')